<a href="https://colab.research.google.com/github/zw2788/LocalMinimaConstruction/blob/main/DwrtXGradientW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from typing import Tuple

import numpy as np
import pandas as pd

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from IPython.display import Image

In [2]:
# parameters for the first layer
W_0 = np.array([[1.05954587,-0.03749863],[-0.05625762,1.09518945]])
b = np.array([[-0.050686],[-0.06894291]])

# parameters for the second layer

V_0 = np.array([[3.76921058,-3.72139955]])
c = np.array([[-0.0148436]])

In [3]:
def forward_prop(
    X_raw: np.array,
    W_0: np.array,
    b: np.array,
    V_0: np.array,
    c: np.array,
) -> Tuple:
    """Performs the forward propagation of the given NN."""
    # Note the NN structure is passed in from outside.
    Z_1 = X_raw @ W_0.T
    A_1 = sigmoid(Z_1 + b.T)

    Z_2 = A_1 @ V_0.T
    A_2 = Y = sigmoid(Z_2 + c.T)

    return A_2, Z_2, A_1, Z_1

In [ ]:
def gradient_magnitude(weights):
    # Compute the gradient of the loss with respect to the weights
    # Return the magnitude of this gradient
    pass

def sample_points_around(center, radius, num_samples):
    # Generate points in the vicinity of 'center' within a 'radius'
    # Return these points
    pass

def average_gradient_magnitude(center, radius, num_samples):
    sampled_points = sample_points_around(center, radius, num_samples)
    magnitudes = [gradient_magnitude(point) for point in sampled_points]
    return sum(magnitudes) / len(magnitudes)

# Optimization loop
for iteration in range(max_iterations):
    # Adjust your input data or parameters to minimize the average gradient magnitude
    # This could involve computing the derivative of this average with respect to your inputs
    pass
